# [Module 1.7] DDP 훈련

본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 아래는 세이지메이커의 어떤 피쳐도 사용하지 않고, PyTorch 만을 사용해서 훈련 합니다.

# PyTorch CIFAR-10 local training  



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cnn-cifar10"

role = sagemaker.get_execution_role()

### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [3]:
inputs = sagemaker_session.upload_data(path="../data", bucket=bucket, key_prefix="data/cifar10")
print("s3 inputs: ", inputs)

s3 inputs:  s3://sagemaker-us-east-1-057716757052/data/cifar10


# Construct a script for training 
- epoch 10 , 20
    - 각각 테스트 정확도 55.2, 62.9


In [4]:
import os
import subprocess

instance_type = "local_gpu"
# instance_type="ml.p3dn.24xlarge",

job_name ='cifar10-horovod'

In [6]:
from sagemaker.pytorch import PyTorch



estimator = PyTorch(
    entry_point="train_ddp.py",    
    source_dir='source',    
    base_job_name = job_name,
    role=role,
    framework_version="1.8.1",
    py_version="py36",
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type= instance_type,
    # sagemaker_session=sagemaker_session,
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
    debugger_hook_config=False,
)
estimator.fit({"training" : inputs})

Creating 1gdw5xuaoo-algo-1-u2fhb ... 
Creating 1gdw5xuaoo-algo-1-u2fhb ... done
Attaching to 1gdw5xuaoo-algo-1-u2fhb
1gdw5xuaoo-algo-1-u2fhb | 2021-07-09 02:43:18,004 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
1gdw5xuaoo-algo-1-u2fhb | 2021-07-09 02:43:18,085 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
1gdw5xuaoo-algo-1-u2fhb | 2021-07-09 02:43:18,088 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel
1gdw5xuaoo-algo-1-u2fhb | 2021-07-09 02:43:18,088 sagemaker_pytorch_container.training INFO     Invoking user training script.
1gdw5xuaoo-algo-1-u2fhb | 2021-07-09 02:43:18,280 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
1gdw5xuaoo-algo-1-u2fhb | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
1gdw5xuaoo-algo-1-u2fhb | Collecting torch==1.6.0
1gdw5xuaoo-algo-1-u2fhb |   Downloading torch-1.6.0-cp36-cp36m-manylinux1_x86_64.whl (7

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1gdw5xuaoo-algo-1-u2fhb | Collecting torchvision==0.7.0
1gdw5xuaoo-algo-1-u2fhb |   Downloading torchvision-0.7.0-cp36-cp36m-manylinux1_x86_64.whl (5.9 MB)
     |████████████████████████████████| 5.9 MB 60.1 MB/s eta 0:00:01
1gdw5xuaoo-algo-1-u2fhb | Collecting torchsummary==1.5.1
1gdw5xuaoo-algo-1-u2fhb |   Downloading torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
1gdw5xuaoo-algo-1-u2fhb | Collecting sagemaker_inference==1.5.5
1gdw5xuaoo-algo-1-u2fhb |   Downloading sagemaker_inference-1.5.5.tar.gz (20 kB)
1gdw5xuaoo-algo-1-u2fhb | Requirement already satisfied: numpy in /opt/conda/lib/python3.6/site-packages (from torch==1.6.0->-r requirements.txt (line 1)) (1.19.1)
1gdw5xuaoo-algo-1-u2fhb | Requirement already satisfied: future in /opt/conda/lib/python3.6/site-packages (from torch==1.6.0->-r requirements.txt (line 1)) (0.18.2)
1gdw5xuaoo-algo-1-u2fhb | Requirement already satisfied: pillow>=4.1.1 in /opt/conda/lib/python3.6/site-packages (from torchvision==0.7.0->-r requirements.txt 

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpjtkrmuww/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

In [17]:
# from sagemaker.pytorch import PyTorch

# cifar10_estimator = PyTorch(
#     entry_point="train_horovod.py",    
#     source_dir='source',    
#     base_job_name = job_name,
#     role=role,
#     framework_version='1.6.0',
#     py_version='py3',
#     train_instance_count=1,
#     train_instance_type=instance_type,
#     hyperparameters={"epochs": 3, 
#                      'lr': 0.01,
#                      'batch-size': 64,
#                      "backend": "gloo",                     
#                     },    
# )
# cifar10_estimator.fit({"training" : inputs})

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Creating d0my0entk3-algo-1-2e2l1 ... 
Creating d0my0entk3-algo-1-2e2l1 ... done
Attaching to d0my0entk3-algo-1-2e2l1
d0my0entk3-algo-1-2e2l1 | 2021-06-08 04:34:43,412 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
d0my0entk3-algo-1-2e2l1 | 2021-06-08 04:34:43,456 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
d0my0entk3-algo-1-2e2l1 | 2021-06-08 04:34:43,459 sagemaker_pytorch_container.training INFO     Invoking user training script.
d0my0entk3-algo-1-2e2l1 | 2021-06-08 04:34:43,614 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
d0my0entk3-algo-1-2e2l1 | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
d0my0entk3-algo-1-2e2l1 | Requirement already satisfied: torch==1.6.0 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (1.6.0)
d0my0entk3-algo-1-2e2l1 | Requirement already satisfied: torchvision==0.7.0 in /opt/conda/lib/python3

In [ ]:
# horovod_artifact_path = cifar10_estimator.model_data
# print("horovod_artifact_path: ", horovod_artifact_path)


# %store horovod_artifact_path

In [ ]:
# ! aws s3 ls {horovod_artifact_path} --recursive